In [1]:

from pyspark.sql import SparkSession

import pyspark.sql.functions as F
import json
from datetime import datetime,timedelta
import os

env = "production"
provider = os.environ.get("CLOUD_PROVIDER", "aws")
# provider = "aws"
# provider = "gcp"
fs = "gs" if provider == "gcp" else "s3"


Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
25,application_1652123500543_0027,pyspark,idle,Link,Link,wenfeng,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# load data

In [2]:
detect_time = datetime.now() - timedelta(hours=1)
date_day = detect_time.strftime("%Y-%m-%d")
date_hour = detect_time.strftime("%H")

date_day = "2022-04-28"
date_hour = "*"
date_hour = "000"
band= "24"
app_name = "zero-clients_{band}".format(band=band)


s3_path = "{fs}://mist-data-science-dev/wenfeng/{app_name}/dt={date}/hr={hr}/*.parquet".format(fs=fs, app_name=app_name, date=date_day, hr=date_hour)
print(s3_path)

df = spark.read.parquet(s3_path)

# df = df.filter("num_wlans>0")

df.printSchema()

# df = df.filter(F.col("band")==24)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

s3://mist-data-science-dev/wenfeng/zero-clients_24/dt=2022-04-28/hr=000/*.parquet
root
 |-- org_id: string (nullable = true)
 |-- site_id: string (nullable = true)
 |-- id: string (nullable = true)
 |-- hostname: string (nullable = true)
 |-- firmware_version: string (nullable = true)
 |-- model: string (nullable = true)
 |-- band: string (nullable = true)
 |-- counts: long (nullable = true)
 |-- channels: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- re_init_first: integer (nullable = true)
 |-- re_init_last: integer (nullable = true)
 |-- max_num_clients: integer (nullable = true)
 |-- tx_errors_max: integer (nullable = true)
 |-- tx_phy_err_max: integer (nullable = true)
 |-- tx_phy_err_min: integer (nullable = true)
 |-- tx_phy_err_avg: double (nullable = true)
 |-- interrupt_stats_tx_bcn_succ_max: integer (nullable = true)
 |-- interrupt_stats_tx_bcn_succ_min: integer (nullable = true)
 |-- bcn_per_wlan_max: float (nullable = true)
 |-- bcn_per_wlan

In [3]:
df.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

194844

In [4]:
df.select("band").groupBy("band").count().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+------+
|band| count|
+----+------+
|  24|194844|
+----+------+

In [5]:
%%pretty
df.filter("noise_floor_high").show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

org_id,site_id,id,hostname,firmware_version,model,band,counts,channels,re_init_first,re_init_last,max_num_clients,tx_errors_max,tx_phy_err_max,tx_phy_err_min,tx_phy_err_avg,interrupt_stats_tx_bcn_succ_max,interrupt_stats_tx_bcn_succ_min,bcn_per_wlan_max,bcn_per_wlan_std,bcn_per_wlan_min,bcn_per_wlan,num_wlans,noise_floor_max,noise_floor_avg,noise_floor_min,utilization_non_wifi_max,utilization_non_wifi_avg,utilization_non_wifi_min,bcn_per_wlan_drop_count,noise_floor_high_count,utilization_non_wifi_high_count,tx_error_high_count,rx_error_high_count,tx_phy_err_high_count,num_channel,reinits,bcn_per_wlan_drop_frac,noise_floor_high_frac,utilization_non_wifi_high_frac,tx_phy_err_high_frac,tx_error_high_frac,rx_error_high_frac,zero_client,bcn_per_wlan_drop,noise_floor_high,utilization_non_wifi_high,tx_phy_err_high,tx_error_high,rx_error_high
ec66b2e6-7108-494...,cfeaeac2-2eb5-4a9...,d4-20-b0-42-6d-0e,AP19-2-bvscl-ca,apfw-0.6.19227-fr...,AP43-US,24,1410,[11],0,0,1,513,0,0,0.0,3031,2848,606.2,9.697659920194258,569.6,597.0449625651041,5,-54,-63.212765957446805,-70,0.2930397689342499,0.12383021606621167,0.09429920464754105,0,1089,21,44,0,0,1,0,0.0,0.7723404255319148,0.014893617021276596,0.0,0.031205673758865248,0.0,false,false,true,false,false,false,false
ec66b2e6-7108-494...,cfeaeac2-2eb5-4a9...,d4-20-b0-c2-0e-8b,AP13-3-bv.scl-ca,apfw-0.10.24046-j...,AP43E-US,24,1411,"[1, 11]",0,0,0,524,0,0,0.0,3030,2732,606.0,14.216157012498643,546.4,587.141886018338,5,-52,-57.838412473423105,-67,0.20319399237632751,0.16576142905875355,0.1391810029745102,0,1409,4,0,0,0,2,0,0.0,0.9985825655563431,0.002834868887313962,0.0,0.0,0.0,true,false,true,false,false,false,false
61e61d9d-5395-4b6...,50529a56-6804-472...,d4-20-b0-41-e0-28,APSMC105,apfw-0.8.21602-ho...,AP43-US,24,1408,[11],0,0,6,746,0,0,0.0,606,581,606.0,9.250213588232727,581.0,598.5014204545455,1,-53,-71.90056818181819,-86,0.3320065140724182,0.06546390402565355,0.001404597423970...,0,410,105,93,0,0,1,0,0.0,0.2911931818181818,0.07457386363636363,0.0,0.06605113636363637,0.0,false,false,true,false,false,false,false
cd01b1e2-7952-4d9...,40552e7f-5804-4c0...,d4-20-b0-c1-5f-bd,tysbx-30f-ap08s,apfw-0.8.21116-ho...,AP43-WW,24,1407,[9],0,0,1,11169,0,0,0.0,2424,2186,606.0,9.590919142957226,546.5,597.1794598436389,4,-46,-67.26652452025586,-90,0.2846129834651947,0.09382046384481395,0.0526450015604496,0,559,4,6,0,0,1,0,0.0,0.39729921819474057,0.002842928216062...,0.0,0.004264392324093...,0.0,false,false,true,false,false,false,false
ec66b2e6-7108-494...,cfeaeac2-2eb5-4a9...,d4-20-b0-45-0c-c6,AP105-2-BN.SCL-CA,apfw-0.8.21602-ho...,AP43-US,24,1409,[1],0,0,1,127,0,0,0.0,3030,2841,606.0,10.36995221660582,568.2,596.0147612810642,5,-52,-61.78353442157559,-71,0.22912926971912384,0.196876349634867,0.155518040060997,0,1241,569,11,0,0,1,0,0.0,0.8807665010645848,0.4038325053229241,0.0,0.007806955287437899,0.0,false,false,true,true,false,false,false
ec66b2e6-7108-494...,cfeaeac2-2eb5-4a9...,d4-20-b0-c2-17-3c,AP206-2-bv.scl-ca,apfw-0.10.24046-j...,AP43E-US,24,1409,"[1, 11]",0,0,0,520,0,0,0.0,3027,1530,605.4,23.297726400704146,306.0,584.2364789151909,5,-53,-57.834634492547906,-63,0.25651103258132935,0.11290428417849152,0.07203079760074615,17,1409,15,5,0,0,2,0,0.0120652945351313,1.0,0.0106458481192335,0.0,0.003548616039744...,0.0,true,false,true,false,false,false,false
61e61d9d-5395-4b6...,b9bf9da7-ccde-444...,5c-5b-35-53-a1-e2,APWKC122,apfw-0.8.21602-ho...,AP43E-US,24,1410,[6],0,0,6,23,0,0,0.0,606,580,606.0,9.338078742475217,580.0,598.1085106382978,1,-49,-72.94893617021276,-94,0.1679539531469345,0.02678062253506154,0.012603958137333393,0,466,0,13,0,0,1,0,0.0,0.3304964539007092,0.0,0.0,0.009219858156028368,0.0,false,false,true,false,false,false,false
38a18d4d-1623-498...,8e3fdfe0-9900-430...,5c-5b-35-d0-6c-70,ATATEBTAPIPKGC00305,apfw-0.8.21602-ho...,AP43-WW,24,1406,"[6, 11]",0,1,0,37,0,0,0.0,1212,0,606.0,230.8968770795787,0.0,108.90789473684211,2,-48,-58.30938833570413,-86,0.5868931412696838,0.06910184869159731,0.0,1151,1150,1

In [6]:

test_cols = ["band", "zero_client", "bcn_per_wlan_drop", "noise_floor_high", "utilization_non_wifi_high",
             "tx_phy_err_high", "tx_error_high", "rx_error_high"]

df_g = df.select(test_cols).groupBy(test_cols).count().orderBy(test_cols)
df_g.show(df_g.count())


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+-----------+-----------------+----------------+-------------------------+---------------+-------------+-------------+------+
|band|zero_client|bcn_per_wlan_drop|noise_floor_high|utilization_non_wifi_high|tx_phy_err_high|tx_error_high|rx_error_high| count|
+----+-----------+-----------------+----------------+-------------------------+---------------+-------------+-------------+------+
|  24|      false|            false|           false|                    false|          false|        false|         true|    66|
|  24|      false|            false|           false|                    false|          false|         true|        false| 22583|
|  24|      false|            false|           false|                    false|          false|         true|         true|    40|
|  24|      false|            false|           false|                    false|           true|        false|        false|     7|
|  24|      false|            false|           false|                    false|    

In [ ]:
# tx_error_high rx_error_high
df_errs = df.filter("tx_error_high and rx_error_high")
df_errs.count()

In [7]:
q0 = "not zero_client and bcn_per_wlan_drop and not noise_floor_high and not utilization_non_wifi_high and not tx_phy_err_high and not tx_error_high and not rx_error_high"   # beacon drop and with clients, no

q1 = "zero_client and not bcn_per_wlan_drop and not noise_floor_high and not utilization_non_wifi_high and not tx_phy_err_high and not tx_error_high and not rx_error_high"   # zero-client and  asymptomatic





FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
%%pretty
df.filter(q0).show()

cols=["max_num_clients", "bcn_per_wlan", "noise_floor_avg", "utilization_non_wifi_avg", "tx_errors_max", "tx_phy_err_avg"]
df.filter(q0).select(cols).summary().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

org_id,site_id,id,hostname,firmware_version,model,band,counts,channels,re_init_first,re_init_last,max_num_clients,tx_errors_max,tx_phy_err_max,tx_phy_err_min,tx_phy_err_avg,interrupt_stats_tx_bcn_succ_max,interrupt_stats_tx_bcn_succ_min,bcn_per_wlan_max,bcn_per_wlan_std,bcn_per_wlan_min,bcn_per_wlan,num_wlans,noise_floor_max,noise_floor_avg,noise_floor_min,utilization_non_wifi_max,utilization_non_wifi_avg,utilization_non_wifi_min,bcn_per_wlan_drop_count,noise_floor_high_count,utilization_non_wifi_high_count,tx_error_high_count,rx_error_high_count,tx_phy_err_high_count,num_channel,reinits,bcn_per_wlan_drop_frac,noise_floor_high_frac,utilization_non_wifi_high_frac,tx_phy_err_high_frac,tx_error_high_frac,rx_error_high_frac,zero_client,bcn_per_wlan_drop,noise_floor_high,utilization_non_wifi_high,tx_phy_err_high,tx_error_high,rx_error_high
0f2adb1c-8843-448...,49b61d8a-8c93-471...,d4-20-b0-81-aa-68,mamsf-ap16,apfw-0.9.22801-il...,AP61-US,24,1412,[11],0,0,1,8,0,0,0.0,606,573,202.0,3.2818198351638532,191.0,198.10623222977853,3,-78,-89.86118980169972,-92,0.24372997879981995,0.038331419761786,0.006501949392259121,1412,0,4,0,0,0,1,0,1.0,0.0,0.0028328611898017,0.0,0.0,0.0,false,true,false,false,false,false,false
138e97d4-4a17-42e...,ff87b456-6674-438...,ac-23-16-10-10-99,AHS-720A,apfw-0.8.21602-ho...,AP32-US,24,1408,[1],0,0,6,2586403,0,0,0.0,4414775,583,700.0,134.8486809089679,291.5,513.9410511363636,2,-84,-98.8828125,-101,0.2108488231897354,0.022118729347725588,0.002407638588920...,397,0,4,20,20,0,1,0,0.2819602272727273,0.0,0.002840909090909091,0.0,0.014204545454545454,0.014204545454545454,false,true,false,false,false,false,false
16342635-98c2-4cf...,64824246-2f53-4c1...,5c-5b-35-2f-10-8a,MW33K Corridor C3...,apfw-0.8.21602-ho...,AP61-WW,24,1410,[6],0,0,3,51,0,0,0.0,606,570,202.0,3.221315033190381,190.0,197.7513003410177,3,-83,-89.41560283687943,-92,0.09209711104631424,0.02833396740429473,0.007879256270825863,1410,0,0,0,0,0,1,0,1.0,0.0,0.0,0.0,0.0,0.0,false,true,false,false,false,false,false
1e43492c-0620-406...,4d23b93d-e9e2-4de...,d4-20-b0-83-1f-01,zb-edge-ap335.net...,apfw-0.8.21602-ho...,AP61-WW,24,1406,[6],0,0,2,32,0,0,0.0,1202,562,300.5,4.689683966256774,140.5,148.9571479374111,4,-77,-87.38406827880512,-92,0.22456704080104828,0.090139713870947,0.01604910008609295,1406,0,16,6,1,0,1,0,1.0,0.0,0.011379800853485065,0.0,0.004267425320056899,7.112375533428165E-4,false,true,false,false,false,false,false
2ff4992c-c62b-11e...,79e2c8b9-39c9-4e4...,d4-dc-09-23-c4-2e,BHQ-AP0405,apfw-0.6.19227-fr...,AP43-US,24,1410,[11],0,0,1,6691,0,0,0.0,1818,1151,606.0,98.4743550072671,383.66666,514.323168512439,3,-73,-88.65390070921985,-92,0.13391192257404327,0.028967245777239956,8.109514019452035E-4,587,0,0,27,2,0,1,0,0.41631205673758864,0.0,0.0,0.0,0.019148936170212766,0.001418439716312...,false,true,false,false,false,false,false
444d11ee-ce73-43d...,7ff65d72-dfde-499...,d4-20-b0-82-96-80,GNS2454_MA103,apfw-0.10.24125-j...,AP32-WW,24,1412,[11],0,0,3,1986,0,0,0.0,1821,1164,607.0,90.1106936434098,388.0,538.204202343992,3,-88,-97.12747875354107,-100,0.2283957600593567,0.03876092771553664,0.007144721690565348,412,0,3,22,1,0,1,0,0.29178470254957506,0.0,0.002124645892351275,0.0,0.015580736543909348,7.08215297450425E-4,false,true,false,false,false,false,false
72351385-5211-42b...,49713171-2f4c-4a3...,d4-dc-09-2b-e7-94,P110-E2,apfw-0.10.24125-j...,AP43E-US,24,1410,[6],0,0,9,12392,0,0,0.0,1818,1144,606.0,95.14962126239489,381.33334,526.2794327106882,3,-86,-90.14397163120567,-91,0.15298643708229065,0.04602242642485187,0.002155696973204...,500,0,0,147,6,0,1,0,0.3546099290780142,0.0,0.0,0.0,0.10425531914893617,0.00425531914893617,false,true,false,false,false,false,false
7e37ab7a-f873-424...,866808ba-1ff2-4e9...,5c-5b-35-2f-14-ef,B3 - Ramp Going t...,apfw-0.10.24125-j...,AP61-WW,24,1413,[11],0,0,4,352,0,0,0.0,605,564,86.42857,1.4531955104884153,80.57143,84.19745221310107,7,-78,-88.59164897381459,-92,0.12290295958518982,0.05086542012648471,0.014022200368344784,1413,0,0

In [9]:
%%pretty
df.filter(q1).show()

cols=["max_num_clients", "bcn_per_wlan", "noise_floor_avg", "utilization_non_wifi_avg", "tx_errors_max", "tx_phy_err_avg"]
df.filter(q1).select(cols).summary().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

org_id,site_id,id,hostname,firmware_version,model,band,counts,channels,re_init_first,re_init_last,max_num_clients,tx_errors_max,tx_phy_err_max,tx_phy_err_min,tx_phy_err_avg,interrupt_stats_tx_bcn_succ_max,interrupt_stats_tx_bcn_succ_min,bcn_per_wlan_max,bcn_per_wlan_std,bcn_per_wlan_min,bcn_per_wlan,num_wlans,noise_floor_max,noise_floor_avg,noise_floor_min,utilization_non_wifi_max,utilization_non_wifi_avg,utilization_non_wifi_min,bcn_per_wlan_drop_count,noise_floor_high_count,utilization_non_wifi_high_count,tx_error_high_count,rx_error_high_count,tx_phy_err_high_count,num_channel,reinits,bcn_per_wlan_drop_frac,noise_floor_high_frac,utilization_non_wifi_high_frac,tx_phy_err_high_frac,tx_error_high_frac,rx_error_high_frac,zero_client,bcn_per_wlan_drop,noise_floor_high,utilization_non_wifi_high,tx_phy_err_high,tx_error_high,rx_error_high
121b3d71-e029-45b...,8e43fffb-05a8-4bc...,d4-dc-09-20-f7-b7,HC-5_AP-13,apfw-0.7.20277-gi...,AP43-US,24,1408,[6],0,0,0,13,0,0,0.0,607,582,607.0,9.198911889462108,582.0,598.8430397727273,1,-72,-87.10085227272727,-90,0.17324607074260712,0.0473863041704383,0.01112095545977354,0,0,0,0,0,0,1,0,0.0,0.0,0.0,0.0,0.0,0.0,true,false,false,false,false,false,false
2397322b-bf83-4f3...,3127ca54-7996-496...,d4-dc-09-20-d9-76,AP A,apfw-0.8.21602-ho...,AP43-US,24,1097,[1],0,0,0,0,0,0,0.0,79245027,580,700.0,9.655374793404306,580.0,599.1130355515041,1,-64,-80.13673655423884,-86,0.14099308848381042,0.06779299672173443,0.0262393020093441,0,1,0,0,1,0,1,0,0.0,9.115770282588879E-4,0.0,0.0,0.0,9.115770282588879E-4,true,false,false,false,false,false,false
280fb5e6-38f3-4a8...,d0ee61c2-eefe-43b...,d4-20-b0-20-ee-48,,apfw-0.9.22325-il...,AP63-US,24,1406,"[1, 6]",0,0,0,1,0,0,0.0,606,581,606.0,8.922298756174618,581.0,599.5803698435277,1,-80,-88.89260312944523,-94,0.2511746287345886,0.1447617673655577,0.08663026988506317,0,0,32,0,0,0,2,0,0.0,0.0,0.02275960170697013,0.0,0.0,0.0,true,false,false,false,false,false,false
34b2bf34-59e4-4dc...,26bc1a80-fb00-44a...,ac-23-16-81-05-d1,480-AP,apfw-0.8.21602-ho...,AP43-US,24,1406,[11],0,0,0,85,0,0,0.0,607,580,607.0,8.890324671627138,580.0,599.4516358463727,1,-77,-85.27524893314367,-87,0.11098434776067734,0.005746988425990853,5.314212785378913E-7,0,0,0,0,0,0,1,0,0.0,0.0,0.0,0.0,0.0,0.0,true,false,false,false,false,false,false
3831255c-e5f3-4b3...,0e539687-2721-4e9...,5c-5b-35-ae-20-3f,WAP1-A3-AE203F-IAD10,apfw-0.8.21602-ho...,AP41-US,24,1410,[6],0,0,0,24072,0,0,0.0,606,561,606.0,10.688507147937827,561.0,593.1843971631206,3,-76,-87.65390070921985,-92,0.20046277344226837,0.0868068768702289,0.04810842126607895,0,0,2,0,0,0,1,0,0.0,0.0,0.001418439716312...,0.0,0.0,0.0,true,false,false,false,false,false,false
38a18d4d-1623-498...,28dfeafe-f4b5-485...,d4-20-b0-41-cf-ca,CAONBURAPMBRL1FL263,apfw-0.8.21626-ho...,AP43-WW,24,1410,[6],0,0,0,10661,0,0,0.0,1212,1158,606.0,9.502769278608756,579.0,597.8652482269504,2,-72,-84.0645390070922,-87,0.12033627182245255,0.023233501490945302,0.013239786028862,0,0,0,6,0,0,1,0,0.0,0.0,0.0,0.0,0.00425531914893617,0.0,true,false,false,false,false,false,false
38a18d4d-1623-498...,4f95a795-8378-493...,d4-20-b0-44-7d-d8,USILCSTAPIWRHSEST29,apfw-0.8.21602-ho...,AP43-US,24,1408,[1],0,0,0,67,0,0,0.0,1213,1158,606.5,9.081386217943658,579.0,598.9868607954545,2,-75,-83.67755681818181,-85,0.16208495199680328,0.10245036173992875,0.053029466420412064,0,0,0,0,0,0,1,0,0.0,0.0,0.0,0.0,0.0,0.0,true,false,false,false,false,false,false
38a18d4d-1623-498...,7736197a-529a-42d...,d4-dc-09-22-2c-77,CNCNST8APIFLRM00101,apfw-0.8.21602-ho...,AP43-WW,24,1408,[11],0,0,0,22,0,0,0.0,1212,1155,606.0,9.057074211882338,577.5,598.9098011363636,2,-67,-83.40198863636364,-86,0.24852658808231354,0.04643139151911865,0.010658531449735165,0,0,7,0,0,0,1,0,0.0,0.0,0.004971590909090909,0.0,0.0,0.0,true,false,false,false,false,false,false
38a18d4d-1623-498...,aed44c88-1e4f-498...,d4-20-b0-43-ac-cd,RURUKLNAPIFLRM00202,apfw-0.8.21602-ho...,AP43-WW,24,1407,"[6, 11]",0,0,0,12,0,0,0.0,1213,1150,606.5,9.2130077723493

In [10]:
cols = ["count", "min", "75%", "90%", "95%", "99%", "max"]
df.select("counts", "bcn_per_wlan_drop_count", "noise_floor_high_count", "tx_phy_err_high_count", "utilization_non_wifi_high_count")\
    .summary(cols).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+------+-----------------------+----------------------+---------------------+-------------------------------+
|summary|counts|bcn_per_wlan_drop_count|noise_floor_high_count|tx_phy_err_high_count|utilization_non_wifi_high_count|
+-------+------+-----------------------+----------------------+---------------------+-------------------------------+
|  count|194844|                 194844|                194844|               194844|                         194844|
|    min|     1|                      0|                     0|                    0|                              0|
|    75%|  1410|                      0|                     0|                    0|                             95|
|    90%|  1412|                    367|                     0|                    0|                            653|
|    95%|  1413|                    587|                     0|                    0|                           1090|
|    99%|  1440|                   1410|                

In [11]:
# df= df.withColumn("bcn_per_wlan_drop_frac", F.col("bcn_per_wlan_drop_count")/F.col("counts"))\
#     .withColumn("noise_floor_high_frac", F.col("noise_floor_high_count")/F.col("counts"))\
#     .withColumn("tx_phy_err_high_frac", F.col("tx_phy_err_high_count")/F.col("counts"))\
#     .withColumn("utilization_non_wifi_high_frac", F.col("utilization_non_wifi_high_count")/F.col("counts"))


summary_cols = ["count", "min", "75%", "90%", "95%", "99%", "max"]
df.filter("bcn_per_wlan_drop_frac>0")\
    .select("bcn_per_wlan_drop_frac", "noise_floor_high_frac", "tx_phy_err_high_frac", "utilization_non_wifi_high_frac").summary(summary_cols).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+----------------------+---------------------+--------------------+------------------------------+
|summary|bcn_per_wlan_drop_frac|noise_floor_high_frac|tx_phy_err_high_frac|utilization_non_wifi_high_frac|
+-------+----------------------+---------------------+--------------------+------------------------------+
|  count|                 27043|                27043|               27043|                         27043|
|    min|  6.939625260235947E-4|                  0.0|                 0.0|                           0.0|
|    75%|    0.5209368346344926|                  0.0|                 0.0|          0.026203966005665724|
|    90%|                   1.0|                  0.0|                 0.0|            0.4657631954350927|
|    95%|                   1.0| 7.102272727272727E-4|7.092198581560284E-4|            0.7981520966595593|
|    99%|                   1.0|   0.7851063829787234|  0.0028388928317956|                           1.0|
|    max|                   1.0|     

In [12]:
# df= df.withColumn("bcn_per_wlan_drop", F.col("bcn_per_wlan_drop_count")>600)\
#     .withColumn("noise_floor_high", F.col("noise_floor_high_count")>600)\
#     .withColumn("tx_phy_err_high", F.col("tx_phy_err_high_count")>600)\
#     .withColumn("utilization_non_wifi_high", F.col("utilization_non_wifi_high_count")>600)
#     .withColumn("tx_errors", F.col("tx_errors_max")>1)

# df.select("bcn_per_wlan_drop", "noise_floor_high", "tx_phy_err_high", "utilization_non_wifi_high")\
# .groupBy("bcn_per_wlan_drop", "noise_floor_high", "tx_phy_err_high", "utilization_non_wifi_high").count().show()


df.select("bcn_per_wlan_drop", "noise_floor_high", "tx_phy_err_high", "utilization_non_wifi_high")\
    .groupBy("bcn_per_wlan_drop", "noise_floor_high", "tx_phy_err_high", "utilization_non_wifi_high").count()\
    .orderBy("bcn_per_wlan_drop", "noise_floor_high", "tx_phy_err_high", "utilization_non_wifi_high")\
    .show(200)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+----------------+---------------+-------------------------+------+
|bcn_per_wlan_drop|noise_floor_high|tx_phy_err_high|utilization_non_wifi_high| count|
+-----------------+----------------+---------------+-------------------------+------+
|            false|           false|          false|                    false|139030|
|            false|           false|          false|                     true| 33552|
|            false|           false|           true|                    false|    30|
|            false|           false|           true|                     true|     8|
|            false|            true|          false|                    false|   741|
|            false|            true|          false|                     true|   216|
|            false|            true|           true|                    false|     1|
|             true|           false|          false|                    false| 19403|
|             true|           false|          false|  

In [13]:
%%pretty
df.filter("not bcn_per_wlan_drop and noise_floor_high").show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

org_id,site_id,id,hostname,firmware_version,model,band,counts,channels,re_init_first,re_init_last,max_num_clients,tx_errors_max,tx_phy_err_max,tx_phy_err_min,tx_phy_err_avg,interrupt_stats_tx_bcn_succ_max,interrupt_stats_tx_bcn_succ_min,bcn_per_wlan_max,bcn_per_wlan_std,bcn_per_wlan_min,bcn_per_wlan,num_wlans,noise_floor_max,noise_floor_avg,noise_floor_min,utilization_non_wifi_max,utilization_non_wifi_avg,utilization_non_wifi_min,bcn_per_wlan_drop_count,noise_floor_high_count,utilization_non_wifi_high_count,tx_error_high_count,rx_error_high_count,tx_phy_err_high_count,num_channel,reinits,bcn_per_wlan_drop_frac,noise_floor_high_frac,utilization_non_wifi_high_frac,tx_phy_err_high_frac,tx_error_high_frac,rx_error_high_frac,zero_client,bcn_per_wlan_drop,noise_floor_high,utilization_non_wifi_high,tx_phy_err_high,tx_error_high,rx_error_high
ec66b2e6-7108-494...,cfeaeac2-2eb5-4a9...,d4-20-b0-42-6d-0e,AP19-2-bvscl-ca,apfw-0.6.19227-fr...,AP43-US,24,1410,[11],0,0,1,513,0,0,0.0,3031,2848,606.2,9.697659920194258,569.6,597.0449625651041,5,-54,-63.212765957446805,-70,0.2930397689342499,0.12383021606621167,0.09429920464754105,0,1089,21,44,0,0,1,0,0.0,0.7723404255319148,0.014893617021276596,0.0,0.031205673758865248,0.0,false,false,true,false,false,false,false
ec66b2e6-7108-494...,cfeaeac2-2eb5-4a9...,d4-20-b0-c2-0e-8b,AP13-3-bv.scl-ca,apfw-0.10.24046-j...,AP43E-US,24,1411,"[1, 11]",0,0,0,524,0,0,0.0,3030,2732,606.0,14.216157012498643,546.4,587.141886018338,5,-52,-57.838412473423105,-67,0.20319399237632751,0.16576142905875355,0.1391810029745102,0,1409,4,0,0,0,2,0,0.0,0.9985825655563431,0.002834868887313962,0.0,0.0,0.0,true,false,true,false,false,false,false
61e61d9d-5395-4b6...,50529a56-6804-472...,d4-20-b0-41-e0-28,APSMC105,apfw-0.8.21602-ho...,AP43-US,24,1408,[11],0,0,6,746,0,0,0.0,606,581,606.0,9.250213588232727,581.0,598.5014204545455,1,-53,-71.90056818181819,-86,0.3320065140724182,0.06546390402565355,0.001404597423970...,0,410,105,93,0,0,1,0,0.0,0.2911931818181818,0.07457386363636363,0.0,0.06605113636363637,0.0,false,false,true,false,false,false,false
cd01b1e2-7952-4d9...,40552e7f-5804-4c0...,d4-20-b0-c1-5f-bd,tysbx-30f-ap08s,apfw-0.8.21116-ho...,AP43-WW,24,1407,[9],0,0,1,11169,0,0,0.0,2424,2186,606.0,9.590919142957226,546.5,597.1794598436389,4,-46,-67.26652452025586,-90,0.2846129834651947,0.09382046384481395,0.0526450015604496,0,559,4,6,0,0,1,0,0.0,0.39729921819474057,0.002842928216062...,0.0,0.004264392324093...,0.0,false,false,true,false,false,false,false
ec66b2e6-7108-494...,cfeaeac2-2eb5-4a9...,d4-20-b0-45-0c-c6,AP105-2-BN.SCL-CA,apfw-0.8.21602-ho...,AP43-US,24,1409,[1],0,0,1,127,0,0,0.0,3030,2841,606.0,10.36995221660582,568.2,596.0147612810642,5,-52,-61.78353442157559,-71,0.22912926971912384,0.196876349634867,0.155518040060997,0,1241,569,11,0,0,1,0,0.0,0.8807665010645848,0.4038325053229241,0.0,0.007806955287437899,0.0,false,false,true,true,false,false,false
ec66b2e6-7108-494...,cfeaeac2-2eb5-4a9...,d4-20-b0-c2-17-3c,AP206-2-bv.scl-ca,apfw-0.10.24046-j...,AP43E-US,24,1409,"[1, 11]",0,0,0,520,0,0,0.0,3027,1530,605.4,23.297726400704146,306.0,584.2364789151909,5,-53,-57.834634492547906,-63,0.25651103258132935,0.11290428417849152,0.07203079760074615,17,1409,15,5,0,0,2,0,0.0120652945351313,1.0,0.0106458481192335,0.0,0.003548616039744...,0.0,true,false,true,false,false,false,false
61e61d9d-5395-4b6...,b9bf9da7-ccde-444...,5c-5b-35-53-a1-e2,APWKC122,apfw-0.8.21602-ho...,AP43E-US,24,1410,[6],0,0,6,23,0,0,0.0,606,580,606.0,9.338078742475217,580.0,598.1085106382978,1,-49,-72.94893617021276,-94,0.1679539531469345,0.02678062253506154,0.012603958137333393,0,466,0,13,0,0,1,0,0.0,0.3304964539007092,0.0,0.0,0.009219858156028368,0.0,false,false,true,false,false,false,false
3831255c-e5f3-4b3...,9a18957a-c129-404...,5c-5b-35-52-1e-17,WAP1-A16-521E17-M...,apfw-0.6.18841-fr...,AP43-WW,24,1410,[1],0,0,0,114,0,0,0.0,2424,2263,606.0,9.96502642058219,565.75,596.4962765957447,4,-44,-60.07659574468085,-77,0.4450933337211609,0.43658312814878236,0.4280417859554

In [14]:
%%pretty
df.filter("not bcn_per_wlan_drop and utilization_non_wifi_high").show()


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

org_id,site_id,id,hostname,firmware_version,model,band,counts,channels,re_init_first,re_init_last,max_num_clients,tx_errors_max,tx_phy_err_max,tx_phy_err_min,tx_phy_err_avg,interrupt_stats_tx_bcn_succ_max,interrupt_stats_tx_bcn_succ_min,bcn_per_wlan_max,bcn_per_wlan_std,bcn_per_wlan_min,bcn_per_wlan,num_wlans,noise_floor_max,noise_floor_avg,noise_floor_min,utilization_non_wifi_max,utilization_non_wifi_avg,utilization_non_wifi_min,bcn_per_wlan_drop_count,noise_floor_high_count,utilization_non_wifi_high_count,tx_error_high_count,rx_error_high_count,tx_phy_err_high_count,num_channel,reinits,bcn_per_wlan_drop_frac,noise_floor_high_frac,utilization_non_wifi_high_frac,tx_phy_err_high_frac,tx_error_high_frac,rx_error_high_frac,zero_client,bcn_per_wlan_drop,noise_floor_high,utilization_non_wifi_high,tx_phy_err_high,tx_error_high,rx_error_high
1f1dcc2c-a7f8-4bc...,b5f34fc3-2d63-445...,d4-20-b0-80-05-50,AP A,apfw-0.8.21602-ho...,AP41-US,24,1410,[1],0,0,0,2,0,0,0.0,606,573,606.0,9.77712114641124,573.0,595.7340425531914,1,-67,-82.93617021276596,-92,0.35017403960227966,0.1728783459898005,0.08411790430545807,0,0,573,0,0,0,1,0,0.0,0.0,0.40638297872340423,0.0,0.0,0.0,true,false,false,true,false,false,false
20101562-d4f8-41a...,07354274-5658-41b...,5c-5b-35-0f-26-d5,AP0433,apfw-0.5.17562-el...,AP41-WW,24,1440,[11],0,0,11,60,0,0,0.0,613,532,613.0,5.049818935388678,532.0,575.2548611111112,2,-71,-82.51875,-92,0.3863060474395752,0.16339021555355027,0.037499841302633286,0,0,534,1,0,0,1,0,0.0,0.0,0.37083333333333335,0.0,6.944444444444445E-4,0.0,false,false,false,true,false,false,false
2a28c8e5-b65f-44a...,e77718a1-701d-4fa...,d4-dc-09-00-fc-49,WS085900fc49,apfw-0.5.17562-el...,AP41-US,24,1412,[1],0,0,0,18,3,0,0.002124645892351275,606,568,606.0,10.175866031377339,568.0,591.9440509915014,2,-70,-80.30524079320114,-90,0.33482295274734497,0.2570943059056744,0.18857161700725555,0,0,1410,0,0,1,1,0,0.0,0.0,0.9985835694050992,7.08215297450425E-4,0.0,0.0,true,false,false,true,false,false,false
353c73c7-3ba9-4ca...,63b5aabc-c27b-46f...,5c-5b-35-ce-3b-29,DRL-2-AP41-48,apfw-0.8.21602-ho...,AP41-US,24,1411,[1],0,0,2,1333,1,0,7.087172218284905E-4,606,567,606.0,9.875953738982622,567.0,594.0170092133238,2,-72,-80.16725726435152,-88,0.280435174703598,0.20219614948155945,0.11756104975938797,0,0,567,60,40,0,1,0,0.0,0.0,0.40184266477675407,0.0,0.042523033309709427,0.028348688873139617,false,false,false,true,false,false,false
39b04356-4cc3-41c...,e39e4738-b9fd-45b...,d4-20-b0-00-b6-03,DC86AP200,apfw-0.8.21602-ho...,AP41E-US,24,1409,[1],0,0,2,40,0,0,0.0,606,563,606.0,10.152323618662635,563.0,593.8495386799149,3,-71,-82.06316536550746,-92,0.32121115922927856,0.17590991319325733,0.05380631610751152,0,0,604,495,11,0,1,0,0.0,0.0,0.42867281760113557,0.0,0.35131298793470545,0.007806955287437899,false,false,false,true,false,true,false
4f7ce014-2148-4d4...,5f430b2f-7a4c-482...,ac-23-16-40-fd-8e,00509.ape.drivethru1,apfw-0.8.21602-ho...,AP32E-US,24,1410,[1],0,0,1,30,0,0,0.0,3000,2853,600.0,9.444044333628673,570.6,588.8080847773991,5,-73,-87.34751773049645,-97,0.2858127951622009,0.1814059631040333,0.10940130800008774,0,0,489,1,0,0,1,0,0.0,0.0,0.3468085106382979,0.0,7.092198581560284E-4,0.0,false,false,false,true,false,false,false
4f7ce014-2148-4d4...,60fa6602-4e49-4c9...,d4-20-b0-20-9c-27,04102.ape.menuboard,apfw-0.8.21602-ho...,AP63E-US,24,1408,[6],0,0,1,12,0,0,0.0,3029,2873,605.8,9.354603014417501,574.6,594.4531251300465,5,-64,-78.68110795454545,-89,0.31242579221725464,0.19464364120027644,0.04279722645878792,0,1,799,1,1,0,1,0,0.0,7.102272727272727E-4,0.5674715909090909,0.0,7.102272727272727E-4,7.102272727272727E-4,false,false,false,true,false,false,false
4f7ce014-2148-4d4...,739d7588-5b1e-4a6...,5c-5b-35-53-b1-55,03675.ape.menuboard,apfw-0.8.21602-ho...,AP43E-US,24,1414,[1],0,0,4,126,0,0,0.0,3633,2858,605.5,40.768292591863286,476.33334,570.1073786669439,6,-62,-75.37835926449787,-86,0.36188629269599915,0.23953606735235225,0.09844670444726944,221,8,1278,14,1,0,1,0,0.156294200848656

In [15]:
%%pretty
df.filter(" bcn_per_wlan_drop and not (noise_floor_high or tx_phy_err_high or utilization_non_wifi_high)")\
    .select("bcn_per_wlan_drop_frac", "noise_floor_high_frac", "tx_phy_err_high_frac", "utilization_non_wifi_high_frac", "noise_floor_max", "tx_phy_err_max", "utilization_non_wifi_max", "tx_errors_max").summary(summary_cols).show()


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

summary,bcn_per_wlan_drop_frac,noise_floor_high_frac,tx_phy_err_high_frac,utilization_non_wifi_high_frac,noise_floor_max,tx_phy_err_max,utilization_non_wifi_max,tx_errors_max
count,19403,19403,19403,19403,19403,19403,19403,19403
min,0.25017717930545713,0.0,0.0,0.0,-98,0,0.0,0
75%,0.5418439716312057,0.0,0.0,0.001417434443656981,-73,0,0.2160169929265976,350
90%,1.0,0.0,0.0,0.017743080198722498,-68,0,0.28897127509117126,1666
95%,1.0,7.092198581560284E-4,0.0,0.05315379163713678,-64,1,0.33131855726242065,7027
99%,1.0,0.007092198581560...,0.001417434443656981,0.18696883852691218,-57,15,0.40678510069847107,342560
max,1.0,0.24397163120567375,0.19098712446351931,0.24883359253499224,-40,36261818,1361.0994873046875,512033770


In [16]:
def get_correlation_matrix_from_df(df_input, input_cols=[], vector_col = "corr_features"):
    from pyspark.ml.feature import VectorAssembler
    from pyspark.ml.stat import Correlation

    assembler = VectorAssembler(inputCols=input_cols,
                                outputCol=vector_col)
    myGraph_vector = assembler.transform(df_input).select(vector_col)
    matrix = Correlation.corr(myGraph_vector, vector_col)
    matrix = Correlation.corr(myGraph_vector, vector_col).collect()[0][0]
    corrmatrix = matrix.toArray().tolist()
    corrmatrix = [[round(x1, 4) for x1 in x ] for x in corrmatrix]


    print(corrmatrix)
    df_corr = spark.createDataFrame(corrmatrix, input_cols)
    df_corr.show()
    return df_corr

input_cols = ["bcn_per_wlan_drop", "noise_floor_high", "tx_phy_err_high", "utilization_non_wifi_high"]
df_corr = get_correlation_matrix_from_df(df, input_cols)

# from pyspark.ml.feature import VectorAssembler
# from pyspark.ml.stat import Correlation

# vector_col = "corr_features"
# input_Cols = ["bcn_per_wlan_drop", "noise_floor_high", "tx_phy_err_high", "utilization_non_wifi_high"]

# assembler = VectorAssembler(inputCols=input_Cols, 
#                             outputCol=vector_col)

# myGraph_vector = assembler.transform(df).select(vector_col)
# # matrix = Correlation.corr(myGraph_vector, vector_col)
# matrix = Correlation.corr(myGraph_vector, vector_col).collect()[0][0]
# corrmatrix = matrix.toArray().tolist()
# corrmatrix = [[round(x1, 4) for x1 in x ] for x in corrmatrix]

# print(corrmatrix)
# df_corr = spark.createDataFrame(corrmatrix,input_Cols)
# df_corr.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[[1.0, 0.0141, 0.047, -0.0946], [0.0141, 1.0, 0.006, 0.0106], [0.047, 0.006, 1.0, -0.0041], [-0.0946, 0.0106, -0.0041, 1.0]]
+-----------------+----------------+---------------+-------------------------+
|bcn_per_wlan_drop|noise_floor_high|tx_phy_err_high|utilization_non_wifi_high|
+-----------------+----------------+---------------+-------------------------+
|              1.0|          0.0141|          0.047|                  -0.0946|
|           0.0141|             1.0|          0.006|                   0.0106|
|            0.047|           0.006|            1.0|                  -0.0041|
|          -0.0946|          0.0106|        -0.0041|                      1.0|
+-----------------+----------------+---------------+-------------------------+

In [17]:
# matrix

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
%%pretty
# s3_t = "s3://mist-data-science-dev/wenfeng/zero-clients/dt=2022-04-22/hr=20/part-00038-b3b8967c-a15f-43f4-a435-56ac4827374b-c000.snappy.parquet"
# df_t = spark.read.parquet(s3_t)
# df_t.show()

# df_t.select("counts", "bcn_per_wlan_drop_count", "noise_floor_high_count", "tx_phy_err_high_count", "utilization_non_wifi_high_count").summary().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
%%pretty
df.filter(df.bcn_per_wlan_drop_count.isNull()).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

org_id,site_id,id,hostname,firmware_version,model,band,counts,channels,re_init_first,re_init_last,max_num_clients,tx_errors_max,tx_phy_err_max,tx_phy_err_min,tx_phy_err_avg,interrupt_stats_tx_bcn_succ_max,interrupt_stats_tx_bcn_succ_min,bcn_per_wlan_max,bcn_per_wlan_std,bcn_per_wlan_min,bcn_per_wlan,num_wlans,noise_floor_max,noise_floor_avg,noise_floor_min,utilization_non_wifi_max,utilization_non_wifi_avg,utilization_non_wifi_min,bcn_per_wlan_drop_count,noise_floor_high_count,utilization_non_wifi_high_count,tx_error_high_count,rx_error_high_count,tx_phy_err_high_count,num_channel,reinits,bcn_per_wlan_drop_frac,noise_floor_high_frac,utilization_non_wifi_high_frac,tx_phy_err_high_frac,tx_error_high_frac,rx_error_high_frac,zero_client,bcn_per_wlan_drop,noise_floor_high,utilization_non_wifi_high,tx_phy_err_high,tx_error_high,rx_error_high


In [20]:
%%pretty
df.filter(df.noise_floor_high_count.isNull()).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

org_id,site_id,id,hostname,firmware_version,model,band,counts,channels,re_init_first,re_init_last,max_num_clients,tx_errors_max,tx_phy_err_max,tx_phy_err_min,tx_phy_err_avg,interrupt_stats_tx_bcn_succ_max,interrupt_stats_tx_bcn_succ_min,bcn_per_wlan_max,bcn_per_wlan_std,bcn_per_wlan_min,bcn_per_wlan,num_wlans,noise_floor_max,noise_floor_avg,noise_floor_min,utilization_non_wifi_max,utilization_non_wifi_avg,utilization_non_wifi_min,bcn_per_wlan_drop_count,noise_floor_high_count,utilization_non_wifi_high_count,tx_error_high_count,rx_error_high_count,tx_phy_err_high_count,num_channel,reinits,bcn_per_wlan_drop_frac,noise_floor_high_frac,utilization_non_wifi_high_frac,tx_phy_err_high_frac,tx_error_high_frac,rx_error_high_frac,zero_client,bcn_per_wlan_drop,noise_floor_high,utilization_non_wifi_high,tx_phy_err_high,tx_error_high,rx_error_high


In [21]:
%%pretty
df.filter(df.bcn_per_wlan_drop.isNull()).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

org_id,site_id,id,hostname,firmware_version,model,band,counts,channels,re_init_first,re_init_last,max_num_clients,tx_errors_max,tx_phy_err_max,tx_phy_err_min,tx_phy_err_avg,interrupt_stats_tx_bcn_succ_max,interrupt_stats_tx_bcn_succ_min,bcn_per_wlan_max,bcn_per_wlan_std,bcn_per_wlan_min,bcn_per_wlan,num_wlans,noise_floor_max,noise_floor_avg,noise_floor_min,utilization_non_wifi_max,utilization_non_wifi_avg,utilization_non_wifi_min,bcn_per_wlan_drop_count,noise_floor_high_count,utilization_non_wifi_high_count,tx_error_high_count,rx_error_high_count,tx_phy_err_high_count,num_channel,reinits,bcn_per_wlan_drop_frac,noise_floor_high_frac,utilization_non_wifi_high_frac,tx_phy_err_high_frac,tx_error_high_frac,rx_error_high_frac,zero_client,bcn_per_wlan_drop,noise_floor_high,utilization_non_wifi_high,tx_phy_err_high,tx_error_high,rx_error_high


In [22]:
# from pyspark.sql.types import *

# def bcn_per_wlan(bcn, num_wlans, model=""):
#     bcn_norm = bcn
#     if model.find("AP41")>-1:
#         bcn_norm= bcn if bcn<610 else 700
#     else:
#         bcn_norm = bcn/num_wlans if num_wlans>0 else 0
#     return bcn_norm if bcn_norm<610 else 700
# bcn_per_wlan_norm = F.udf(bcn_per_wlan, FloatType())

# df=df.withColumn("bcn_per_wlan_norm",bcn_per_wlan_norm(F.col("interrupt_stats_tx_bcn_succ_max"), F.col("num_wlans"), F.col("model")) )

# cols = [  "noise_floor_avg", "num_wlans",  "tx_phy_err_avg",  "utilization_non_wifi_avg", "interrupt_stats_tx_bcn_succ_max", "bcn_per_wlan_max", "bcn_per_wlan_norm"]
# df = df.withColumn("noise_floor_high", F.col("noise_floor_avg")>-60)\
#     .withColumn("tx_phy_err_high", F.col("tx_phy_err_avg")>1)\
#     .withColumn("utilization_non_wifi_high", F.col("utilization_non_wifi_max")>0.30)\
#     .withColumn("bcn_per_wlan_drop", F.col("bcn_per_wlan_norm")<500)
    

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
# %%pretty
# df.show()


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
# df_g = df.select("noise_floor_high", "tx_phy_err_high", "utilization_non_wifi_high", "bcn_per_wlan_drop" ).groupBy("noise_floor_high", "tx_phy_err_high", "utilization_non_wifi_high", "bcn_per_wlan_drop").count()

# df_g.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
# df_g = df.filter("bcn_per_wlan_norm<500").select("noise_floor_high", "tx_phy_err_high", "utilization_non_wifi_high", "bcn_per_wlan_drop" ).groupBy("noise_floor_high", "tx_phy_err_high", "utilization_non_wifi_high", "bcn_per_wlan_drop").count()

# df_g.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
# df.count()
df.columns

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['org_id', 'site_id', 'id', 'hostname', 'firmware_version', 'model', 'band', 'counts', 'channels', 're_init_first', 're_init_last', 'max_num_clients', 'tx_errors_max', 'tx_phy_err_max', 'tx_phy_err_min', 'tx_phy_err_avg', 'interrupt_stats_tx_bcn_succ_max', 'interrupt_stats_tx_bcn_succ_min', 'bcn_per_wlan_max', 'bcn_per_wlan_std', 'bcn_per_wlan_min', 'bcn_per_wlan', 'num_wlans', 'noise_floor_max', 'noise_floor_avg', 'noise_floor_min', 'utilization_non_wifi_max', 'utilization_non_wifi_avg', 'utilization_non_wifi_min', 'bcn_per_wlan_drop_count', 'noise_floor_high_count', 'utilization_non_wifi_high_count', 'tx_error_high_count', 'rx_error_high_count', 'tx_phy_err_high_count', 'num_channel', 'reinits', 'bcn_per_wlan_drop_frac', 'noise_floor_high_frac', 'utilization_non_wifi_high_frac', 'tx_phy_err_high_frac', 'tx_error_high_frac', 'rx_error_high_frac', 'zero_client', 'bcn_per_wlan_drop', 'noise_floor_high', 'utilization_non_wifi_high', 'tx_phy_err_high', 'tx_error_high', 'rx_error_high']

In [27]:
df_noise_floor = df.filter("noise_floor_avg>-85")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [28]:
%%spark -o df_noise_floor -n 50000

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
Invalid status code '404' from http://ip-172-31-18-8.us-west-1.compute.internal:8998/sessions/25/statements/29 with error payload: {"msg":"Session '25' not found."}


In [29]:
%%local
# %%local 

import plotly.express as px
# px.histogram(df, x=["bcn_per_wlan_norm"], log_y=True)

px.histogram(df_noise_floor, x=["noise_floor_avg"], log_y=True)

NameError: name 'df_noise_floor' is not defined

In [ ]:
%%spark -o df -n 50000

In [ ]:
%%local
# %%local 

import plotly.express as px
px.histogram(df, x=["bcn_per_wlan"], log_y=True)

# px.histogram(df, x=["noise_floor_avg"], log_y=True)

In [ ]:
%%local
# %%local 

import plotly.express as px
# px.histogram(df, x=["bcn_per_wlan_norm"], log_y=True)

px.histogram(df[df['utilization_non_wifi_avg']<1.0], x=["utilization_non_wifi_avg"], log_y=True)

In [ ]:
%%local
# %%local 

import plotly.express as px
# px.histogram(df, x=["bcn_per_wlan_norm"], log_y=True)

px.histogram(df[df['tx_phy_err_max']>0]["tx_phy_err_max"], log_y=True, log_x=False)

In [ ]:
%%local 

from plotly.subplots import make_subplots
import plotly.graph_objects as go



cols = [  "noise_floor_avg", "num_wlans",  "tx_phy_err_avg",  "utilization_non_wifi_avg", "interrupt_stats_tx_bcn_succ_max", "bcn_per_wlan_max", "bcn_per_wlan_norm"]


def plot(df, title=""):
    fig = make_subplots(rows=6, cols=2,
                       subplot_titles=('noise_floor_avg',
                                       'tx_phy_err_avg',
                                       'utilization_non_wifi_avg',
                                       'bcn_per_wlan_norm',
                                        'bcn_per_wlan_norm vs nf',
                                       'bcn_per_wlan_norm vs phy_err',
                                      'bcn_per_wlan_norm vs non_wif',
                                       'nf vs non_wifi'))
    
    fig.add_trace(
        go.Histogram(x = df["noise_floor_avg"]),
        row=1, col=1
    )
    
    fig.add_trace(
        go.Histogram(x = df["tx_phy_err_avg"]),
        row=1, col=2
    )
        
    fig.add_trace(
        go.Histogram(x = df["utilization_non_wifi_avg"]),
        row=2, col=1
    )
    
    fig.add_trace(
        go.Histogram(x = df["bcn_per_wlan"]),
        row=2, col=2
    )
     
    fig.add_trace(
        go.Histogram(x = df["bcn_per_wlan_drop_count"]),
        row=3, col=1
    )
    
    fig.add_trace(
        go.Histogram(x = df["noise_floor_high_count"]),
        row=3, col=2
    )
    fig.add_trace(
        go.Histogram(x = df["tx_phy_err_high_count"]),
        row=4, col=1
    )
    fig.add_trace(
        go.Histogram(x = df["utilization_non_wifi_high_count"]),
        row=4, col=2
    )
    
    fig.add_trace(
        go.Histogram2dContour(x=df["bcn_per_wlan"], y=df["noise_floor_avg"]),
        row=5, col=1
    )
    
    fig.add_trace(
        go.Histogram2dContour(x=df["bcn_per_wlan"], y=df["tx_phy_err_avg"]),
        row=5, col=2
    )

    fig.add_trace(
        go.Histogram2dContour(x=df["bcn_per_wlan"], y=df["utilization_non_wifi_avg"]),
        row=6, col=1
    )
        
    fig.add_trace(
        go.Histogram2dContour(x=df["noise_floor_avg"], y=df["utilization_non_wifi_avg"]),
        row=6, col=2
    )
    

    
    fig.layout['barmode'] = 'overlay'
    fig.update_layout(height=2000, width=1200, title_text="{}".format(title), xaxis_type="log", yaxis_type="log")
    fig.show()
    
plot(df)







In [ ]:
%%local 
import plotly.express as px

px.scatter(df, x="bcn_per_wlan", y="noise_floor_avg")
# px.scatter(df_ap_capacity_stats_test, x="avg_nclients", y="util_ap", log_x=True)

In [ ]:
%%local 
px.scatter(df, x="bcn_per_wlan", y="noise_floor_avg")

In [ ]:
# %%local 
# px.scatter(df, x="tx_errors_max", y="rx_errors_max")

In [ ]:
%%local 

from plotly.subplots import make_subplots
import plotly.graph_objects as go



cols = [  "noise_floor_avg", "num_wlans",  "tx_phy_err_avg",  "utilization_non_wifi_avg", "interrupt_stats_tx_bcn_succ_max", "bcn_per_wlan_max", "bcn_per_wlan_norm"]



def plot(df, title=""):
    fig = make_subplots(rows=2, cols=2,
                       subplot_titles=(
                                        'bcn_per_wlan_norm vs nf',
                                       'bcn_per_wlan_norm vs phy_err',
                                      'bcn_per_wlan_norm vs non_wif',
                                       'nf vs non_wifi'))
    
    fig.add_trace(
        go.Scatter(x=df["bcn_per_wlan"], y=df["noise_floor_avg"], mode='markers'),
        row=1, col=1
    )
    fig.add_trace(
        go.Scatter(x=df["bcn_per_wlan"], y=df["tx_phy_err_avg"], mode='markers'),
        row=1, col=2
    )

    df1 = df[df["utilization_non_wifi_avg"]<1.0]
    fig.add_trace(
        go.Scatter(x=df1["bcn_per_wlan"], y=df1["utilization_non_wifi_avg"], mode='markers'),
        row=2, col=1
    )
        
    fig.add_trace(
        go.Scatter(x=df1["noise_floor_avg"], y=df1["utilization_non_wifi_avg"], mode='markers'),
        row=2, col=2
    )
    

    fig.layout['barmode'] = 'overlay'
    fig.update_layout(height=2000, width=1200, title_text="{}".format(title))
    #     fig.update_layout(height=2000, width=1200, title_text="{}".format(title), xaxis_type="log", yaxis_type="log")
    fig.show()
    
plot(df)





